In [2]:
import splink.comparison_library as cl
import pandas as pd
import numpy as np
import json
import os, sys
from splink.exploratory import completeness_chart
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets
from splink_tools import *
from pprint import pprint 

def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")

Current working directory:  /home/ubuntu/OrgSync


### Check for duplicate ids in datasets

Cleaned
```
len(gtr_df): 69067
len(cordis_df): 36508
uniqu in id gtr: 69067
uniqu in id cordis: 5398
```

raw w/ cordis = just one dataset (FP7, organisation.json)
```
len(gtr_df): 69067
len(cordis_df): 123419
uniqu in id gtr: 69067
uniqu in id cordis: 28177
```

```python
data_dir = os.path.join("data", "raw")
with open(os.path.join(data_dir, "gtr_data.json"), "r", encoding="utf8") as f:
    gtr_json = json.load(f)
with open(os.path.join(data_dir, "uk_data.json"), "r", encoding="utf8") as f:
    cordis_json = json.load(f)

gtr_df = pd.DataFrame(gtr_json)
cordis_df = pd.DataFrame(cordis_json)

print(f"len(gtr_df): {len(gtr_df)}")
print(f"len(cordis_df): {len(cordis_df)}")
print(f"uniqu in id gtr: {len(gtr_df['id'].unique())}")
print(f"uniqu in id cordis: {len(cordis_df['organisationID'].unique())}")

base = os.path.join("data", "raw", "all_scraped")
file_paths = [
    "cordis/2024_07/FP7/organization.json",
    "gtr/scraped/2024_07/organisations.json"
]
with open(os.path.join(base, file_paths[0]), "r", encoding="utf8") as f:
    cordis_json = json.load(f)
with open(os.path.join(base, file_paths[1]), "r", encoding="utf8") as f:
    gtr_json = json.load(f)

cordis_df = pd.DataFrame(cordis_json)
gtr_df = pd.DataFrame(gtr_json)

print(f"len(gtr_df): {len(gtr_df)}")
print(f"len(cordis_df): {len(cordis_df)}")
print(f"uniqu in id gtr: {len(gtr_df['id'].unique())}")
print(f"uniqu in id cordis: {len(cordis_df['organisationID'].unique())}")
```

In [3]:
base_path = os.path.join("data", "splink")
data_path = os.path.join(base_path, "all_data.json")
postcodes_path = os.path.join(base_path, "parsed_postcodes.json")

# load the data
df = pd.read_json(data_path)
df_postcodes = pd.read_json(postcodes_path)
print(f"len(df): {len(df)}")
print(f"len(df_postcodes): {len(df_postcodes)}")
print(f"unique_id in df: {df['unique_id'].nunique()}")
print(f"unique_id in df_postcodes: {df_postcodes['unique_id'].nunique()}")

df.drop_duplicates(inplace=True)
df_postcodes.drop_duplicates(inplace=True)

print(f"len(df): {len(df)}")
print(f"len(df_postcodes): {len(df_postcodes)}")
print(f"unique_id in df: {df['unique_id'].nunique()}")
print(f"unique_id in df_postcodes: {df_postcodes['unique_id'].nunique()}")


def create_unique_identifiers(df, df_postcodes):
    """
    #! This function is no longer needed but kept in case it crops up again.
    The two input datasets contain columns for the same entries in the same order.
    Many entires have the same `unique_id` in both datasets. This function creates a new column
    `global_id` that is unique for each entry in the datasets.

    It then merges the two datasets on this new column.
    """
    assert df["unique_id"].to_list() == df_postcodes["unique_id"].to_list()
    assert df["dataset"].to_list() == df_postcodes["dataset"].to_list()
    
    df["global_id"] = range(len(df))
    df_postcodes["global_id"] = range(len(df_postcodes))
    # convert the global_id to string
    df["global_id"] = df["global_id"].astype(str)
    df_postcodes["global_id"] = df_postcodes["global_id"].astype(str)
    df = df.merge(df_postcodes, on=["global_id", "dataset", "unique_id"], how="left")
    return df

data_unique_ids = df['unique_id'].to_list()
postcodes_unique_ids = df_postcodes['unique_id'].to_list()
data_datasets = df["dataset"].to_list()
postcodes_datasets = df_postcodes["dataset"].to_list()
# check if the unique_ids in the data and postcodes are the same
assert data_unique_ids == postcodes_unique_ids
assert data_datasets == postcodes_datasets

df.head()

len(df): 105573
len(df_postcodes): 105573
unique_id in df: 74508
unique_id in df_postcodes: 74508
len(df): 74508
len(df_postcodes): 74508
unique_id in df: 74508
unique_id in df_postcodes: 74508


,unique_id,name,postcode,dataset
0,1906596,nuclear decommissioning authority nda,ca24 3hu,cordis
1,2123954,south west tourism limited,ex2 5wt,cordis
2,1915686,welsh government,cf10 3nq,cordis
3,2128407,terrasalus limited,le15 9el,cordis
4,2168344,ol pharma partners ltd,sg4 7dp,cordis


In [4]:
df_postcodes.head()

,unique_id,dataset,parsed.postcode,parsed.original,parsed.incode,parsed.outcode,parsed.area,parsed.district,parsed.sub_district,parsed.sector,parsed.unit,parsed.fix_distance,parsed.is_in_ons_postcode_directory
0,1906596,cordis,CA24 3HU,ca24 3hu,3HU,CA24,CA,CA24,None,CA24 3,HU,0.0,1.0
1,2123954,cordis,EX2 5WT,ex2 5wt,5WT,EX2,EX,EX2,None,EX2 5,WT,0.0,0.0
2,1915686,cordis,CF10 3NQ,cf10 3nq,3NQ,CF10,CF,CF10,None,CF10 3,NQ,0.0,1.0
3,2128407,cordis,LE15 9EL,le15 9el,9EL,LE15,LE,LE15,None,LE15 9,EL,0.0,1.0
4,2168344,cordis,SG4 7DP,sg4 7dp,7DP,SG4,SG,SG4,None,SG4 7,DP,0.0,1.0


In [5]:
def pd_create_parsed_postcode_columns(data, postcode_data):
    """
    Marge data dn post_code on unique_id

    Inputs:
    data: pd.DataFrame
    postcode_data: pd.DataFrame
    """

    # merge on index
    merged_data = pd.merge(data, postcode_data, how="left", on="unique_id")

    # data = pd.concat([data, postcode_data], axis=1)
    return merged_data

df = pd_create_parsed_postcode_columns(df, df_postcodes)
df.head()


,unique_id,name,postcode,dataset_x,dataset_y,parsed.postcode,parsed.original,parsed.incode,parsed.outcode,parsed.area,parsed.district,parsed.sub_district,parsed.sector,parsed.unit,parsed.fix_distance,parsed.is_in_ons_postcode_directory
0,1906596,nuclear decommissioning authority nda,ca24 3hu,cordis,cordis,CA24 3HU,ca24 3hu,3HU,CA24,CA,CA24,None,CA24 3,HU,0.0,1.0
1,2123954,south west tourism limited,ex2 5wt,cordis,cordis,EX2 5WT,ex2 5wt,5WT,EX2,EX,EX2,None,EX2 5,WT,0.0,0.0
2,1915686,welsh government,cf10 3nq,cordis,cordis,CF10 3NQ,cf10 3nq,3NQ,CF10,CF,CF10,None,CF10 3,NQ,0.0,1.0
3,2128407,terrasalus limited,le15 9el,cordis,cordis,LE15 9EL,le15 9el,9EL,LE15,LE,LE15,None,LE15 9,EL,0.0,1.0
4,2168344,ol pharma partners ltd,sg4 7dp,cordis,cordis,SG4 7DP,sg4 7dp,7DP,SG4,SG,SG4,None,SG4 7,DP,0.0,1.0


In [6]:
# display df where parsed.postcode is null
df[df["parsed.postcode"].isnull()].head(50)
# df[df[""]]

,unique_id,name,postcode,dataset_x,dataset_y,parsed.postcode,parsed.original,parsed.incode,parsed.outcode,parsed.area,parsed.district,parsed.sub_district,parsed.sector,parsed.unit,parsed.fix_distance,parsed.is_in_ons_postcode_directory
30,1653759,alkane energy uk limited,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
33,1653780,smart moves limited ta city car club,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
34,1653782,mrcmcleanhazel ltd,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
35,1653794,university of the west of england,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
36,1653798,bath north east somerset council,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
37,1653809,powabyke ltd,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
38,1653823,advanced communications information systems ltd,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
39,1653825,first somerset avon ltd,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
40,1653829,advanced transport systems ltd,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
41,1614807,cpl scientific publishing services ltd,None,cordis,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN


In [ ]:
# find all rows with postcode.original == "ectified"
# df[df["parsed.original"] == "ectified"]
df[df["parsed.district"] == "EC1"]
df.head(50)

,unique_id,name,postcode,dataset_x,dataset_y,parsed.postcode,parsed.original,parsed.incode,parsed.outcode,parsed.area,parsed.district,parsed.sub_district,parsed.sector,parsed.unit,parsed.fix_distance,parsed.is_in_ons_postcode_directory
0,1906596,nuclear decommissioning authority nda,ca24 3hu,cordis,cordis,CA24 3HU,ca24 3hu,3HU,CA24,CA,CA24,None,CA24 3,HU,0.0,1.0
1,2123954,south west tourism limited,ex2 5wt,cordis,cordis,EX2 5WT,ex2 5wt,5WT,EX2,EX,EX2,None,EX2 5,WT,0.0,0.0
2,1915686,welsh government,cf10 3nq,cordis,cordis,CF10 3NQ,cf10 3nq,3NQ,CF10,CF,CF10,None,CF10 3,NQ,0.0,1.0
3,2128407,terrasalus limited,le15 9el,cordis,cordis,LE15 9EL,le15 9el,9EL,LE15,LE,LE15,None,LE15 9,EL,0.0,1.0
4,2168344,ol pharma partners ltd,sg4 7dp,cordis,cordis,SG4 7DP,sg4 7dp,7DP,SG4,SG,SG4,None,SG4 7,DP,0.0,1.0
5,1917645,forsite diagnostics ltd,yo41 1lz,cordis,cordis,YO41 1LZ,yo41 1lz,1LZ,YO41,YO,YO41,None,YO41 1,LZ,0.0,1.0
6,1907389,st georges hospital medical school,sw17 0re,cordis,cordis,SW17 0RE,sw17 0re,0RE,SW17,SW,SW17,None,SW17 0,RE,0.0,1.0
7,1909146,genetic alliance uk ltd,n1 3qp,cordis,cordis,N1 3QP,n1 3qp,3QP,N1,N,N1,None,N1 3,QP,0.0,1.0
8,2102071,fetal medicine foundation,w1g 6bg,cordis,cordis,W1G 6BG,w1g 6bg,6BG,W1G,W,W1,W1G,W1G 6,BG,0.0,1.0
9,2142093,centre of the cell,e1 2ab,cordis,cordis,E1 2AB,e1 2ab,2AB,E1,E,E1,None,E1 2,AB,0.0,1.0


In [ ]:
db_api = DuckDBAPI()
# Conver empty lists to null before running...#
completeness_chart(df, db_api=db_api, table_names_for_chart=["dataset"])


alt.LayerChart(...)